# 843. Guess the Word (Very Hard!!!)


This is an interactive problem.

You are given an array of unique strings wordlist where wordlist[i] is 6 letters long, and one word in this list is chosen as secret.

You may call Master.guess(word) to guess a word. The guessed word should have type string and must be from the original list with 6 lowercase letters.

This function returns an integer type, representing the number of exact matches (value and position) of your guess to the secret word. Also, if your guess is not in the given wordlist, it will return -1 instead.

For each test case, you have exactly 10 guesses to guess the word. At the end of any number of calls, if you have made 10 or fewer calls to Master.guess and at least one of these guesses was secret, then you pass the test case.

In [ ]:
def match(self, w1, w2):
    return sum(i == j for i, j in zip(w1, w2))

#### Solution 2.1: Shuffle the Wordlist
I didn't give up the previous idea, it's not that bad.
So I decided to try my luck by shuffling wordlist at the beginning.

Note that it may sound some unreliable,
but actully randomicity is very useful trick in both competition and reality problem.

This method can get accepted but not for sure.
After manualy testing locally,
on Leetcode's test cases set and random test cases set,
it has roughly 70% rate to get accepted.

Time complexity O(N)
Space complexity O(N)


In [31]:
def findSecretWord(self, wordlist: List[str], master: 'Master') -> None:
    random.shuffle(wordlist)
    for i in range(10):
        guess = random.choice(wordlist)
        x = master.guess(guess)
        wordlist = [w for w in wordlist if sum(i == j for i, j in zip(guess, w)) == x] # choose the word at least has the same match with the guessed word

#### Solution 2.2: Guess a Random Word
All words are generated randomly.
So why not we also guess a random word and let it be whatever will be.
This is actually the same as the previous solution.
Though we don't need one more O(N) operation to shuffle the wordlist at first.

Time complexity O(N)
Space complexity O(N)

In [32]:
def findSecretWord(self, wordlist, master):
    for i in range(10):
        guess = random.choice(wordlist)
        x = master.guess(guess)
        wordlist = [w for w in wordlist if sum(i == j for i, j in zip(guess, w)) == x]

#### Solution 3: Minimax
Now we want to try a better solution.
Generally, we will get 0 matches from the master.guess.
As a result, the size of wordlist reduces slowly.

Recall some math here, the possiblity that get 0 matched is:
(25/26) ^ 6 = 79.03%

That is to say, if we make a blind guess,
we have about 80% chance to get 0 matched with the secret word.

To simplify the model,
we're going to assume that,
we will always run into the worst case (get 0 matched).

In this case,
we have 80% chance to eliminate the candidate word
as well as its close words which have at least 1 match.

Additionally, in order to delete a max part of words,
we select a candidate who has a big "family",
(that is, the fewest 0 matched with other words.)
We want to guess a word that can minimum our worst outcome.

So we compare each two words and count their matches.
For each word, we note how many word of 0 matches it gets.
Then we guess the word with minimum words of 0 matches.

In this solution, we apply a minimax idea.
We minimize our worst case,
The worst case is max(the number of words with x matches),
and we assume it equal to "the number of words with 0 matches"

Time complexity O(N^2)
Space complexity O(N)

In [33]:
def findSecretWord(self, wordlist, master):
        x = 0
        while x < 6:
            count = collections.Counter(w1 for w1, w2 in itertools.permutations(wordlist, 2) if self.match(w1, w2) == 0)
            guess = min(wordlist, key=lambda w: count[w])
            x = master.guess(guess)
            wordlist = [w for w in wordlist if self.match(w, guess) == x]

#### Solution 4: Count the Occurrence of Characters
In the previous solution, we compaired each two words.
This make the complexity O(N^2) for each turn.

But actually we don't have to do that.
We just need to count the occurrence for each character on each position.

If we can guess the word that not in the wordlist,
we can guess the word based on the most frequent character on the position.

Here we have to guess a word from the list,
we still can calculate a score of similarity for each word,
and guess the word with highest score.

Time complexity O(N)
Space complexity O(N)

In [35]:
def findSecretWord(self, wordlist, master):
        def match(w1, w2):
            return sum(i == j for i, j in zip(w1, w2))

        n = 0
        while n < 6:
            count = [collections.Counter(w[i] for w in wordlist) for i in xrange(6)]
            guess = max(wordlist, key=lambda w: sum(count[i][c] for i, c in enumerate(w)))
            n = master.guess(guess)
            wordlist = [w for w in wordlist if match(w, guess) == n]

# 362. Design Hit Counter


Design a hit counter which counts the number of hits received in the past 5 minutes (i.e., the past 300 seconds).

Your system should accept a timestamp parameter (in seconds granularity), and you may assume that calls are being made to the system in chronological order (i.e., timestamp is monotonically increasing). Several hits may arrive roughly at the same time.

Implement the HitCounter class:

HitCounter() Initializes the object of the hit counter system.
void hit(int timestamp) Records a hit that happened at timestamp (in seconds). Several hits may happen at the same timestamp.
int getHits(int timestamp) Returns the number of hits in the past 5 minutes from timestamp (i.e., the past 300 seconds).
 



In [ ]:
class HitCounter:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        init_vals = [0]*300
        self.queue = deque(init_vals, maxlen = 300)
        self.oldest = 0

    def hit(self, timestamp: int) -> None:
        """
        Record a hit.
        @param timestamp - The current timestamp (in seconds granularity).
        """
        if timestamp >300:
            if timestamp-300-self.oldest>300:
                init_vals = [0]*300
                self.queue = deque(init_vals, maxlen = 300)
            else:
                for i in range(timestamp-300-self.oldest):
                    self.queue.popleft() 
                    self.queue.append(0) 
            self.oldest = timestamp - 300
            self.queue[-1] +=1
            # print("timestamp:",timestamp, "queue:", self.queue)

        else:
            self.queue[timestamp-1] +=1
            # print('else:',"timestamp:",timestamp, "queue:", self.queue)

        

    def getHits(self, timestamp: int) -> int:
        """
        Return the number of hits in the past 5 minutes.
        @param timestamp - The current timestamp (in seconds granularity).
        """
        result = 0
        if timestamp> 300:
            if timestamp-300-self.oldest>300:
                init_vals = [0]*300
                self.queue = deque(init_vals, maxlen = 300)
            else:
                for i in range(timestamp-300-self.oldest):
                    self.queue.popleft() 
                    self.queue.append(0) 
            self.oldest = timestamp - 300
            # print("timestamp:",timestamp, "queue:", self.queue)

            result = sum(self.queue)
        else:
            for i in range(timestamp):
                result+= self.queue[i]
        return result
        


# Your HitCounter object will be instantiated and called as such:
# obj = HitCounter()
# obj.hit(timestamp)
# param_2 = obj.getHits(timestamp)


# Your HitCounter object will be instantiated and called as such:
# obj = HitCounter()
# obj.hit(timestamp)
# param_2 = obj.getHits(timestamp)

Complexity Analysis

Time Complexity

hit - Since inserting a value in the queue takes place in O(1)O(1) time, hence hit method works in O(1)O(1).

getHits - Assuming a total of nn values present in the queue at a time and the total number of timestamps encountered throughout is NN. In the worst case scenario, we might end up removing all the entries from the queue in getHits method if the difference in timestamp is greater than or equal to 300. Hence in the worst case, a "single" call to the getHits method can take O(n)O(n) time. However, we must notice that each timestamp is processed only twice (first while adding the timestamp in the queue in hit method and second while removing the timestamp from the queue in the getHits method). Hence if the total number of timestamps encountered throughout is NN, the overall time taken by getHits method is O(N)O(N). This results in an amortized time complexity of O(1)O(1) for a single call to getHits method.

Space Complexity: Considering the total timestamps encountered throughout to be NN, the queue can have upto NN elements, hence overall space complexity of this approach is O(N)O(N).



In [36]:
class HitCounter {
    private Queue<Integer> hits; 

    /** Initialize your data structure here. */
    public HitCounter() {
        this.hits = new LinkedList<Integer>();
    }
    
    /** Record a hit.
        @param timestamp - The current timestamp (in seconds granularity). */
    public void hit(int timestamp) {
        this.hits.add(timestamp);
    }
    
    /** Return the number of hits in the past 5 minutes.
        @param timestamp - The current timestamp (in seconds granularity). */
    public int getHits(int timestamp) {
        while (!this.hits.isEmpty()) {
            int diff = timestamp - this.hits.peek();
            if (diff >= 300) this.hits.remove();
            else break;
        }
        return this.hits.size();
    }
}

SyntaxError: invalid syntax (<ipython-input-36-40afbb7a7d1f>, line 1)